In [1]:

from classes.Grid import *


def run_app():
    root = Tk()
    root.geometry("1000x1000")
    GridWindow(root)
    root.mainloop()


if __name__ == '__main__':

    run_app()



Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-1.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-1.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-2.json


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/longtaoliu/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/Users/longtaoliu/PycharmProjects/MLCMS01/src/classes/Grid.py", line 158, in init_setup
    self.load_grid(data)
  File "/Users/longtaoliu/PycharmProjects/MLCMS01/src/classes/Grid.py", line 221, in load_grid
    for row, col in data['pedestrians']:
TypeError: 'NoneType' object is not subscriptable


Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-3.json


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/longtaoliu/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/Users/longtaoliu/PycharmProjects/MLCMS01/src/classes/Grid.py", line 158, in init_setup
    self.load_grid(data)
  File "/Users/longtaoliu/PycharmProjects/MLCMS01/src/classes/Grid.py", line 221, in load_grid
    for row, col in data['pedestrians']:
TypeError: 'NoneType' object is not subscriptable


Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-3.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-4.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-4.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-4.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-4.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-4.json
Loading file from /Users/longtaoliu/PycharmProjects/MLCMS01/src/test2-5.json
